In [2]:
import pandas as pd
import oracledb
from datetime import datetime

# DB credentials
username = "system"
password = "Wwcict@1219@"
dsn = "localhost:1521/xe"

# Load the CSV
csv_path = "../data/sentiment_reviews.csv"  # Update this if needed
df = pd.read_csv(csv_path)

try:
    # Connect to Oracle
    print("Connecting to Oracle...")
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("✅ Connected to Oracle.")

    # Step 1: Insert unique banks into banks table
    bank_name_to_id = {}
    for bank in df["bank"].unique():
        cursor.execute("SELECT id FROM banks WHERE name = :1", [bank])
        result = cursor.fetchone()
        if result:
            bank_id = result[0]
        else:
            cursor.execute("INSERT INTO banks (name) VALUES (:1) RETURNING id INTO :2", [bank, cursor.var(int)])
            bank_id = cursor.getimplicitresults()[0][0]
        bank_name_to_id[bank] = bank_id

    print(f"✅ Inserted {len(bank_name_to_id)} unique banks.")

    # Step 2: Insert reviews
    for _, row in df.iterrows():
        bank_id = bank_name_to_id.get(row["bank"])
        review_text = str(row["review"])
        sentiment_label = row.get("sentiment", "neutral")
        sentiment_score = float(row.get("sentiment_score", 0))
        if "top_keywords" in df.columns and pd.notnull(row.get("top_keywords", None)):
            top_keywords = ", ".join(eval(row["top_keywords"])) if isinstance(row["top_keywords"], str) and row["top_keywords"].startswith("[") else str(row["top_keywords"])
        else:
            top_keywords = ""
        review_date = datetime.strptime(str(row["date"]), "%Y-%m-%d").date()

        cursor.execute("""
            INSERT INTO reviews (
                bank_id, review_text, sentiment_label,
                sentiment_score, top_keywords, review_date
            ) VALUES (:1, :2, :3, :4, :5, :6)
        """, [bank_id, review_text, sentiment_label, sentiment_score, top_keywords, review_date])

    connection.commit()
    print("✅ All reviews inserted into Oracle.")

except oracledb.DatabaseError as e:
    print("❌ Oracle DB error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()
        print("🔒 Connection closed.")


Connecting to Oracle...
✅ Connected to Oracle.
✅ Inserted 3 unique banks.
✅ All reviews inserted into Oracle.
🔒 Connection closed.


 Inserting Banks and Reviews

In [3]:
import pandas as pd
import oracledb
from datetime import datetime

# Oracle DB credentials
username = "system"
password = "Wwcict@1219@"
dsn = "localhost:1521/xe"

# Updated CSV file path
csv_path = "../data/sentiment_reviews.csv"

# Load CSV data
df = pd.read_csv(csv_path)

try:
    # Connect to Oracle DB
    print("🔌 Connecting to Oracle...")
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("✅ Connected to Oracle Database.")

    # Step 1: Insert unique banks into banks table
    bank_name_to_id = {}
    for bank in df["bank"].unique():
        cursor.execute("SELECT id FROM banks WHERE name = :1", [bank])
        result = cursor.fetchone()
        if result:
            bank_id = result[0]
        else:
            cursor.execute("INSERT INTO banks (name) VALUES (:1)", [bank])
            connection.commit()  # Ensure ID is available after insert
            cursor.execute("SELECT id FROM banks WHERE name = :1", [bank])
            bank_id = cursor.fetchone()[0]
        bank_name_to_id[bank] = bank_id

    print(f"🏦 Inserted or confirmed {len(bank_name_to_id)} banks.")

    # Step 2: Insert reviews into reviews table
    for _, row in df.iterrows():
        try:
            bank_id = bank_name_to_id.get(row["bank"])
            review_text = str(row.get("review", ""))
            sentiment_label = row.get("sentiment", "neutral")
            sentiment_score = float(row.get("sentiment_score", 0))

            if "top_keywords" in row and pd.notnull(row["top_keywords"]):
                top_keywords = ", ".join(eval(row["top_keywords"])) if isinstance(row["top_keywords"], str) and row["top_keywords"].startswith("[") else str(row["top_keywords"])
            else:
                top_keywords = ""

            review_date = datetime.strptime(str(row["date"]), "%Y-%m-%d").date()

            cursor.execute("""
                INSERT INTO reviews (
                    bank_id, review_text, sentiment_label,
                    sentiment_score, top_keywords, review_date
                ) VALUES (:1, :2, :3, :4, :5, :6)
            """, [bank_id, review_text, sentiment_label, sentiment_score, top_keywords, review_date])
        except Exception as row_err:
            print(f"⚠️ Skipping row due to error: {row_err}")

    connection.commit()
    print("✅ All reviews imported into Oracle database.")

except oracledb.DatabaseError as e:
    print("❌ Oracle error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()
        print("🔒 Connection closed.")


🔌 Connecting to Oracle...
✅ Connected to Oracle Database.
🏦 Inserted or confirmed 3 banks.
✅ All reviews imported into Oracle database.
🔒 Connection closed.


In [4]:
import oracledb

# Credentials
username = "system"
password = "Wwcict@1219@"
dsn = "localhost:1521/xe"

try:
    # Reconnect to DB
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("✅ Connected for verification query.")

    # Sample query: get 3 negative reviews
    cursor.execute("""
        SELECT review_text, sentiment_label, top_keywords
        FROM reviews
        WHERE sentiment_label = 'negative'
        FETCH FIRST 3 ROWS ONLY
    """)
    for r in cursor.fetchall():
        print(r)

except oracledb.DatabaseError as e:
    print("❌ Oracle DB error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()
        print("🔒 Connection closed after query.")


✅ Connected for verification query.
🔒 Connection closed after query.


In [6]:
import oracledb
import pandas as pd

# Oracle connection settings
username = "system"
password = "Wwcict@1219@"
dsn = "localhost:1521/xe"

try:
    # Connect to Oracle
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    cursor = connection.cursor()
    print("✅ Connected to Oracle Database")

    # Query 1: Count banks
    cursor.execute("SELECT COUNT(*) FROM banks")
    bank_count = cursor.fetchone()[0]
    print(f"🏦 Total banks: {bank_count}")

    # Query 2: Count reviews
    cursor.execute("SELECT COUNT(*) FROM reviews")
    review_count = cursor.fetchone()[0]
    print(f"📝 Total reviews: {review_count}")

    # Query 3: Sample banks
    cursor.execute("SELECT * FROM banks")
    banks_df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
    print("\n📋 Sample Banks Table:")
    display(banks_df)

    # Query 4: Sample reviews (first 5 rows)
    cursor.execute("SELECT * FROM reviews WHERE ROWNUM <= 5")
    reviews_df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
    print("\n📋 Sample Reviews Table:")
    display(reviews_df)

except oracledb.DatabaseError as e:
    print("❌ Oracle error:", e)

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()
        print("🔒 Connection closed.")


✅ Connected to Oracle Database
🏦 Total banks: 3
📝 Total reviews: 8889

📋 Sample Banks Table:


,ID,NAME
0,10,Dashen Bank
1,11,Bank of Abyssinia
2,12,Commercial Bank of Ethiopia



📋 Sample Reviews Table:


,ID,BANK_ID,REVIEW_TEXT,SENTIMENT_LABEL,SENTIMENT_SCORE,TOP_KEYWORDS,REVIEW_DATE
0,1,10,love,1,0.0,None,2025-06-06
1,2,10,መቸሸጠ,0,0.0,None,2025-06-03
2,3,10,wow,1,0.0,None,2025-06-03
3,4,10,gadaa,0,0.0,None,2025-06-01
4,5,10,Massive upgrade from the Amole app.,0,0.0,None,2025-05-31


🔒 Connection closed.


In [10]:
from IPython.display import display, Markdown

report = """
# Detailed Report: Data Insertion into Oracle Database (`insert_db.ipynb`)

## Overview

This notebook documents the process of importing sentiment review data from a CSV file into an Oracle database. The workflow includes reading the data, connecting to the Oracle DB, inserting unique banks, and importing reviews with associated metadata.

---

## Steps Performed

### 1. **Data Loading**
- The CSV file (`../data/sentiment_reviews.csv`) was loaded into a pandas DataFrame (`df`).
- The DataFrame contains 2963 rows and 7 columns, including review text, rating, date, bank name, source, language, and sentiment.

### 2. **Database Connection**
- Used the `oracledb` library to connect to an Oracle XE instance.
- Credentials:
    - Username: `system`
    - Password: (hidden for security)
    - DSN: `localhost:1521/xe`
- Connection and cursor objects were created for executing SQL statements.

### 3. **Inserting Unique Banks**
- Extracted unique bank names from the DataFrame.
- For each bank:
    - Checked if the bank already exists in the `banks` table.
    - If not, inserted the bank and retrieved its generated ID.
- Created a mapping (`bank_name_to_id`) from bank names to their database IDs.
- Example mapping:
    ```python
    {'Dashen Bank': 10, 'Bank of Abyssinia': 11, 'Commercial Bank of Ethiopia': 12}
    ```
- Total banks inserted/confirmed: 3

### 4. **Inserting Reviews**
- Iterated over each row in the DataFrame.
- For each review:
    - Mapped the bank name to its ID.
    - Extracted review text, sentiment label, sentiment score, top keywords (if available), and review date.
    - Inserted the review into the `reviews` table with all relevant fields.
- Committed the transaction after all inserts.

### 5. **Verification and Exploration**
- Queried the database to verify data insertion:
    - Counted total banks and reviews.
    - Displayed sample data from both `banks` and `reviews` tables.
    - Queried for specific reviews (e.g., negative sentiment).
- Used pandas DataFrames (`banks_df`, `reviews_df`) for tabular display in the notebook.

---

## Results

- **Banks Table:**  
    | ID | NAME                        |
    |----|-----------------------------|
    | 10 | Dashen Bank                 |
    | 11 | Bank of Abyssinia           |
    | 12 | Commercial Bank of Ethiopia |

- **Reviews Table:**  
    - Total reviews inserted: 8889 (as per `review_count`)
    - Sample fields: `REVIEW_TEXT`, `SENTIMENT_LABEL`, `TOP_KEYWORDS`, `REVIEW_DATE`, etc.

---

## Error Handling

- Used try-except blocks to catch and report database errors.
- Skipped problematic rows during review insertion and logged the error.

---

## Conclusion

The notebook successfully automated the process of importing sentiment review data into an Oracle database, ensuring referential integrity between banks and reviews, and providing verification steps to confirm successful data migration.
"""

display(Markdown(report))


# Detailed Report: Data Insertion into Oracle Database (`insert_db.ipynb`)

## Overview

This notebook documents the process of importing sentiment review data from a CSV file into an Oracle database. The workflow includes reading the data, connecting to the Oracle DB, inserting unique banks, and importing reviews with associated metadata.

---

## Steps Performed

### 1. **Data Loading**
- The CSV file (`../data/sentiment_reviews.csv`) was loaded into a pandas DataFrame (`df`).
- The DataFrame contains 2963 rows and 7 columns, including review text, rating, date, bank name, source, language, and sentiment.

### 2. **Database Connection**
- Used the `oracledb` library to connect to an Oracle XE instance.
- Credentials:
    - Username: `system`
    - Password: (hidden for security)
    - DSN: `localhost:1521/xe`
- Connection and cursor objects were created for executing SQL statements.

### 3. **Inserting Unique Banks**
- Extracted unique bank names from the DataFrame.
- For each bank:
    - Checked if the bank already exists in the `banks` table.
    - If not, inserted the bank and retrieved its generated ID.
- Created a mapping (`bank_name_to_id`) from bank names to their database IDs.
- Example mapping:
    ```python
    {'Dashen Bank': 10, 'Bank of Abyssinia': 11, 'Commercial Bank of Ethiopia': 12}
    ```
- Total banks inserted/confirmed: 3

### 4. **Inserting Reviews**
- Iterated over each row in the DataFrame.
- For each review:
    - Mapped the bank name to its ID.
    - Extracted review text, sentiment label, sentiment score, top keywords (if available), and review date.
    - Inserted the review into the `reviews` table with all relevant fields.
- Committed the transaction after all inserts.

### 5. **Verification and Exploration**
- Queried the database to verify data insertion:
    - Counted total banks and reviews.
    - Displayed sample data from both `banks` and `reviews` tables.
    - Queried for specific reviews (e.g., negative sentiment).
- Used pandas DataFrames (`banks_df`, `reviews_df`) for tabular display in the notebook.

---

## Results

- **Banks Table:**  
    | ID | NAME                        |
    |----|-----------------------------|
    | 10 | Dashen Bank                 |
    | 11 | Bank of Abyssinia           |
    | 12 | Commercial Bank of Ethiopia |

- **Reviews Table:**  
    - Total reviews inserted: 8889 (as per `review_count`)
    - Sample fields: `REVIEW_TEXT`, `SENTIMENT_LABEL`, `TOP_KEYWORDS`, `REVIEW_DATE`, etc.

---

## Error Handling

- Used try-except blocks to catch and report database errors.
- Skipped problematic rows during review insertion and logged the error.

---

## Conclusion

The notebook successfully automated the process of importing sentiment review data into an Oracle database, ensuring referential integrity between banks and reviews, and providing verification steps to confirm successful data migration.
